# Testing Several Different Prompting Methodologies (via SOLAR 10.7B)


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Upstage/SOLAR-10.7B-Instruct-v1.0")
model = AutoModelForCausalLM.from_pretrained(
    "Upstage/SOLAR-10.7B-Instruct-v1.0",
    device_map="auto",
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## Zero-Shot + Instruction Prompting

- Simply feed the task text to the model and ask for results.


In [4]:
zero_shot = [
    {
        "role": "user",
        "content": "Choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming",
    }
]

prompt = tokenizer.apply_chat_template(
    zero_shot, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
Choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming

### Assistant:
1. Playful
2. Gentle
3. Majestic</s>


## Few-Shot + Instruction Prompting

- Feed a set of high-quality demonstrations, each consisting of both input and desired output, on the target task.
- Likely to feed similar names : use k-NN in the embedding space ([Liu et al., 2021](https://arxiv.org/abs/2101.06804))
- Constructing good example set might be another task.


In [5]:
few_shot = [
    {
        "role": "user",
        "content": """Based on the examples below, choose 3 from the following descriptions that match the given name: Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Do not provide any other information.
        Name: Snow White
        Answer: Playful, Gentle, Majestic
        Name: Snowy
        Answer: Curious, Loyal, Witty
        Name: Snowflake
        Answer: """,
    }
]

prompt = tokenizer.apply_chat_template(
    few_shot, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
Based on the examples below, choose 3 from the following descriptions that match the given name: Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Do not provide any other information.
        Name: Snow White
        Answer: Playful, Gentle, Majestic
        Name: Snowy
        Answer: Curious, Loyal, Witty
        Name: Snowflake
        Answer: 

### Assistant:
Name: Snowflake
Answer: Mysterious, Adventurous, Elegant</s>


## Few-Shot + Instruction Prompting (alternative)

**_Using different tasks as example sets._**

- Repeating the same task as an example is not necessary.


In [6]:
few_shot_alt = [
    {
        "role": "user",
        "content": """Based on the examples below, choose 3 from the following descriptions that match the given name: Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Do not provide any other information.
        Name: Snow
        Answer: White, Bright, Romantic
        Name: Winter
        Answer: Frosty, Cold, Serene
        Name: Christmas
        Answer: Romantic, Cold, Serene
        Name: Snowflake
        Answer: """,
    }
]

prompt = tokenizer.apply_chat_template(
    few_shot_alt, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
Based on the examples below, choose 3 from the following descriptions that match the given name: Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Do not provide any other information.
        Name: Snow
        Answer: White, Bright, Romantic
        Name: Winter
        Answer: Frosty, Cold, Serene
        Name: Christmas
        Answer: Romantic, Cold, Serene
        Name: Snowflake
        Answer: 

### Assistant:
1. Majestic
2. Mysterious
3. Elegant</s>


## Self-Consistency Sampling : [Wang et al., 2022a](https://arxiv.org/pdf/2203.11171.pdf)

- Sample diverse answers through different reasoning paths, then apply majority vote.
- $temperature > 0$


In [7]:
sc_sampling = [
    {
        "role": "user",
        "content": "Choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Now, using self-consistency sampling, describe a fictional or idealized version of Snowflake, incorporating those chosen characteristics into a cohesive and vivid portrayal. Do not provide any other information.",
    }
]

prompt = tokenizer.apply_chat_template(
    sc_sampling, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
Choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Now, using self-consistency sampling, describe a fictional or idealized version of Snowflake, incorporating those chosen characteristics into a cohesive and vivid portrayal. Do not provide any other information.

### Assistant:
Snowflake is a captivating being, embodying the enchanting qualities of elegance, curiosity, and wit. As gentle as a soft winter breeze, Snowflake moves with grace and poise, exuding an air of majesty. This enigmatic creature possesses an insatiable thirst for knowledge, exploring the world around them with a sense of adventure. Their charm is undeniable, drawing others to their charismatic presence. Despite their playful nature, Snowflake remains loyal to their friends and values, making them a trustworthy and delightful companion. Their unique blend of characteristics creates a mes

In [8]:
sc_sampling_alt = [
    {
        "role": "user",
        "content": """By the method below, choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming.
        First, pick the very best description that match name about 20 times, use different reasoning pathway each.
        Then show me the top three most frequently chosen descriptions. Do not provide any other information.""",
    }
]

prompt = tokenizer.apply_chat_template(
    sc_sampling_alt, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
By the method below, choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming.
        First, pick the very best description that match name about 20 times, use different reasoning pathway each.
        Then show me the top three most frequently chosen descriptions. Do not provide any other information.

### Assistant:
To find the top three most frequently chosen descriptions that match the name 'Snowflake', I will analyze the given options using different reasoning pathways.

Reasoning Pathway 1: Winter and Nature Associations
- Snowflakes are delicate and elegant creations of winter.
- They are often associated with gentleness and beauty.
- Choices: Gentle, Elegant

Reasoning Pathway 2: Uniqueness and Individuality
- Snowflakes are unique and no two are alike.
- This can be associated with curiosity and playfulness.
- Choices: Playful, Curious

Reasoning Pathway

## Chain-of-Thought (CoT) : [Wei et al., 2022](https://arxiv.org/pdf/2201.11903.pdf)


In [9]:
cot_zero = [
    {
        "role": "user",
        "content": """Based on the examples below, choose 3 from the following descriptions that match the given name: Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Do not provide any other information.
        Name: Snowflake
        Answer: Let's think step by step.""",
    }
]

prompt = tokenizer.apply_chat_template(
    cot_zero, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
Based on the examples below, choose 3 from the following descriptions that match the given name: Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Do not provide any other information.
        Name: Snowflake
        Answer: Let's think step by step.

### Assistant:
1. Playful
2. Gentle
3. Mysterious (due to the association with snow and its enigmatic nature)</s>


**Citation :**
[Lilian Weng, 2023](https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/)
